In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sb
import statsmodels.api as sm
from workalendar.america import BrazilBankCalendar

In [39]:
###Importando as bases
##Criando colunas:
#detrator / neutro / promotor / [FLAG] Entrega x nps
from numpy import unicode_

base = pd.read_csv(r'../NPS_Bulky/analise_dummy_ou_outros (version 1).csv',sep=';', encoding='ISO-8859-1')
base.shape

dummy = pd.get_dummies(base['3. Sac_pos_entrega'], prefix='sac_pos_soma_').iloc[:, 1:]

#Cria as colunas de soma das dummys
def somase(data_frame):

    colunas = ['Flag Atraso Transporte2','Flag Atraso Fornecedor3','Flag Atraso Coleta4','Flag Atraso Cliente5','[FLAG]Cancelado',
    '[FLAG]Recompra_ff','[FLAG]Erro prazo transporte validacao','[FLAG]Cross','[FLAG]Atraso_criacao_OC','[FLAG]Reprogramacao_OC',
    '[FLAG]Atraso Fornecedor','[FLAG] Entregue','[FLAG] Pratica_indevida','[FLAG] Sac_pos_entrega','[FLAG] Reversa','[FLAG] PJ',
    '[FLAG] Mais de uma entrega','[FLAG]Importacao WMS','[FLAG] Recompra','[FLAG] Entrega_Mista','[FLAG] Virada_Faturamento',
    'FLAG - Infos_encontradas','[FLAG] Atraso limite','[FLAG] Oc sem bip','[FLAG] Insucesso','[FLAG] End. N.Localizado/Ausente',
    '[FLAG] Tempo Habil','[FLAG] Atraso Redespacho','[FLAG] Diarias','[FLAG] Atraso Coletas','[FLAG] Leves','[FLAG] Atraso CD',
    '[FLAG] Recompra assistencia','[FLAG] Recompra Total','[FLAG] Origem Recompra','[FLAG] Atraso Cliente*','FLAG_Parceiro',
    'FLAG_terceiras_parceiras','[FLAG] - Detrator_mais_entrega','[FLAG] - Categorias_detrator','[FLAG] Private_Label','Guide_flag'
                ]
   
    df_retorno = data_frame
    for coluna in colunas:
        try:
            nova_coluna = df_retorno.groupby('pv_pai')[coluna].sum()
            df_retorno = df_retorno.merge(nova_coluna, left_on='pv_pai', right_on='pv_pai',how='left')
            df_retorno.rename(columns={coluna + '_y': coluna + '_soma'}, inplace=True,errors='raise')
            # print('realizado colunas: ', coluna + '_soma')
        except Exception as e:
            print('deu ruim: ', e)
    return df_retorno

#funcao dias
def dia_util(d1, d2):
    cal = BrazilBankCalendar()
    cal.include_ash_wednesday = False
    a = d1
    b = d2
    if pd.isnull(d1) or pd.isnull(d2):
        return np.nan
    else:
        a = pd.to_datetime(d1, infer_datetime_format=True, dayfirst=True)
        b = pd.to_datetime(d2, infer_datetime_format=True, dayfirst=True)
        if b.weekday() in [5, 6]:
            if b > a:
                resp = (cal.get_working_days_delta(a, b)) + 1
            else:
                resp = (cal.get_working_days_delta(a, b))
                resp = resp * -1
        else:
            resp = (cal.get_working_days_delta(a, b))
            if a > b:
                return -resp
    return resp

#flag entrega prometida:

def flag_entrega_prometida(data_limite_cliente, data_compra):
    if dia_util(data_limite_cliente, data_compra) <= 5:
        return int(1)
    else:
        return int(0)

#regra flag aging x nps:
def flag_nps_x_entrega(limite_cliente, data_entregue,qtd_dias_atraso):
    if dia_util(limite_cliente, data_entregue) > qtd_dias_atraso:
        return int(1)
    else:
        return int(0)

    
#Roda a função somase com a base importada
base = somase(base)

#Cria as colunas de detrator, neutro e promotor
def dummy_detra(resposta):
    if resposta <= 6:
        return "1"
    else:
        return "0"

def neutro(resposta):
    if resposta >= 7 and resposta <= 8:
        return "1"
    else:
        return "0"

def promotor (resposta):
    if resposta >= 9:
        return '1'
    else:
        return '0'

#criacao de colunas:
base['detrator'] = base.apply(
        lambda row: dummy_detra(row['customer_answer']), axis=1)

base['neutro'] = base.apply(
lambda row: neutro(row['customer_answer']), axis=1)

base['promotor'] = base.apply(
        lambda row: promotor(row['customer_answer']), axis=1)

############################################################FLAGS PROMOTORAS#####################################
#####flag entrega x nps, mudar na funcao o numero de dias:
base['[FLAG] Entrega_adiantada'] = base.apply(
        lambda row: flag_nps_x_entrega(row['data_limite_cliente'], row['data_entregue'], -3), axis=1)

base['[FLAG] Promessa_de_entrega <=5'] = base.apply(
        lambda row: flag_entrega_prometida(row['data_limite_cliente'], row['data_compra_cliente']), axis=1)


#Drop colunas desnecessárias
base = base.drop(columns=['3. Sac_pos_entrega_s','97 Atraso WMS_s','96 Recompra_s','Reclamacao_cliente_s','Info_Encontradas_s','Coluna34','Coluna4',
'Coluna2','cidade_cliente','pedped','cep','chave_nfm','chave_ped','nf_fornecedor','fornecedor','nf_mi','data_compra_cliente','sku',
'categoria_old','categoria_new','head_categoria','descricao','chave_pedped','pedido_x','origem','destino','tp_t3','data_criacao_oc',
'data_aprovacao','data_limite_fornecedor','data_expedicao','data_limite_cliente','data_limite_coleta','data_limite_transporte','data_bip',
'data_entregue','pz_total_x','nota_nfd','tp_nf','tp_eagle','status_pedido_portal','Atraso Local V1','OC','Motivo_Recompra','prazo_interno_x',
'prazo_interno_y','channel','Virada_FF_VAO','Pesso Juridica','pedido_prazo_fornecedor','pedido_prazo_transportadora','cotacao_prazo_transporte',
'cotacao_prazo_fornecedor','Título Romaneio','Motorista','Roteiro','Prioridade','Codigo_EDI','EDI','Importacao_FF','Atraso Local V2',
'92.Virada_ff_Vao','91.Erro de Prazo Transporte','89.Reversa','1. Depara Transportadora','2. Pratica_indevida','3. Sac_pos_entrega',
'97 Atraso WMS','96 Recompra','Consolidador','Data_resposta','Media_dia_x_resposta','Motivo_sac Entrega','Reclamacao_cliente','chave_TP',
'DePara_TP_Parceiro:','Rank_Regras','[Prazo] aux','Dia Sem. Aprovação','Feriado','Prazo Vendido','Delta prazo cotado',
'Data Limite Cotado','aux Comparação Prazo Total (Cotado x Vendido)','Aux_coleta','Prazo_interno','Prazo_x_exp','CEP2','Pedido_original_Recompra',
'Data_Limite_para_Recompra','nova_data_limite','Aging_cliente','data-entrega','Chave_espaco_2','Fornecedor_ocr_desc',
'Guide','Categoria_nova_estrutura','Cabe_casa',
'Yes_casa','% detrator_categorias','qtd. categoria_detrator','1 cat','2 cat','3 cat','4 cat','5 cat','Loja',
'[compra] Foi fácil realizar sua compra?','Estado_destino_sigla','Semana','Mês',
'[pagamento] Como foi a sua experiência com relação ao pagamento do produto adquirido?','Como foi sua experiência enquanto esperava o seu produto?','[entrega] Você ficou satisfeito(a) com a entrega?',
'[produto] O produto atendeu às suas expectativas?','[atendimento] Você ficou satisfeito com nosso atendimento?','[entrega]] A embalagem chegou com problemas (rasgada, danificada, molhada...)',
'[entrega]] A entrega foi feita fora do prazo','[[entrega]] Achei o prazo de entrega muito longo.]',
'[[entrega]] Achei o valor do frete muito alto.]','[[entrega]] Não foi possível agendar a entrega.]',
'[atendimento]] (aberta) Outro. Por favor, especifique:]','(aberta) Como foi sua experiência enquanto esperava o seu produto?',
'[compra]] (aberta) Foi fácil realizar sua compra?]','[entrega]] (aberta) Por favor, nos conte mais sobre a sua experiência.',
'[entrega]] (aberta) Você ficou satisfeito(a) com a entrega?','[pagamento]] (aberta) Visando melhorar essa experiência, você pode contar quais d',
'[produto]] (aberta) Por favor, nos conte mais sobre a sua resposta','Atraso_Transp_s','Flag Atraso Fornecedor_s','Flag Atraso Coleta_s','Flag Atraso Cliente_s',
'[FLAG]Cancelado_S','[FLAG]Recompra_ff_S','[FLAG]Erro prazo transporte validacao_S','96.Atraso_importação_WMS_S',
'[FLAG]Cross_S','[FLAG]Atraso_criacao_OC_S','[FLAG]Reprogramacao_OC_S','92.Virada_ff_Vao_S','91.Erro de Prazo Transporte_S',
'[FLAG]Atraso Fornecedor_S','89.Reversa_S','NPS_NOTA_S','[FLAG] Entregue_S','[FLAG] Pratica_indevida_s','[FLAG] Sac_pos_entrega_s',
'[FLAG] Reversa_s','[FLAG] PJ_s','[FLAG] Mais de uma entrega_s','[FLAG]Importacao WMS_s','[FLAG] Recompra_s',
'[FLAG] Entrega_Mista_s','[FLAG] Virada_Faturamento_s','FLAG - Infos_encontradas_s','[FLAG] Atraso limite_s',
'[FLAG] Oc sem bip_s','[FLAG] Insucesso_s','[FLAG] End. N.Localizado/Ausente_s','[FLAG] Tempo Habil_s',
'[FLAG] Atraso Redespacho_s','[FLAG] Diarias_s','Flag Atraso Transporte2_x','Flag Atraso Fornecedor3_x',
'Flag Atraso Coleta4_x','Flag Atraso Cliente5_x','[FLAG]Cancelado_x','[FLAG]Recompra_ff_x','[FLAG]Erro prazo transporte validacao_x',
'96.Atraso_importação_WMS','[FLAG]Cross_x','[FLAG]Atraso_criacao_OC_x','[FLAG]Reprogramacao_OC_x','[FLAG]Atraso Fornecedor_x','NPS_NOTA',
'[FLAG] Entregue_x','[FLAG] Pratica_indevida_x','[FLAG] Sac_pos_entrega_x','[FLAG] Reversa_x','[FLAG] PJ_x','[FLAG] Mais de uma entrega_x',
'[FLAG]Importacao WMS_x','[FLAG] Recompra_x','[FLAG] Entrega_Mista_x','[FLAG] Virada_Faturamento_x','Info_Encontradas',
'FLAG - Infos_encontradas_x','[FLAG] Atraso limite_x','[FLAG] Oc sem bip_x','[FLAG] Insucesso_x',
'[FLAG] End. N.Localizado/Ausente_x','[FLAG] Tempo Habil_x','[FLAG] Atraso Redespacho_x',
'[FLAG] Diarias_x','[FLAG] Atraso Coletas_x','[FLAG] Leves_x','[FLAG] Atraso CD_x','[FLAG] Recompra assistencia_x',
'[FLAG] Recompra Total_x','[FLAG] Origem Recompra_x','[FLAG] Atraso Cliente*_x','FLAG_Parceiro_x','FLAG_terceiras_parceiras_x',
'[FLAG] - Detrator_mais_entrega_x','[FLAG] - Categorias_detrator_x','[FLAG] Private_Label_x','Guide_flag_x',
'[FLAG] Origem Recompra_soma',
'FLAG_terceiras_parceiras_soma',
'tipo_venda','Guide_flag_soma','[FLAG] Entregue_soma', 'created_at','tipo_pedido_nps','[FLAG]Recompra_ff_soma','[FLAG] PJ_soma','FLAG - Infos_encontradas_soma','[FLAG] Recompra assistencia_soma','[FLAG] Recompra Total_soma','[FLAG] Atraso Cliente*_soma','FLAG_Parceiro_soma','[FLAG] - Detrator_mais_entrega_soma','[FLAG] - Categorias_detrator_soma',
])

dummy
base = pd.concat([base, dummy], axis=1)
base.head()

C:\Users\veronica.souza\AppData\Local\Temp\ipykernel_87068\300532503.py:6: DtypeWarning: Columns (90,91,96,98,99,101,102) have mixed types. Specify dtype option on import or set low_memory=False.
  base = pd.read_csv(r'../NPS_Bulky/analise_dummy_ou_outros (version 1).csv',sep=';', encoding='ISO-8859-1')


,customer_answer,pv_pai,Flag Atraso Transporte2_soma,Flag Atraso Fornecedor3_soma,Flag Atraso Coleta4_soma,Flag Atraso Cliente5_soma,[FLAG]Cancelado_soma,[FLAG]Erro prazo transporte validacao_soma,[FLAG]Cross_soma,[FLAG]Atraso_criacao_OC_soma,...,[FLAG] Private_Label_soma,detrator,neutro,promotor,[FLAG] Entrega_adiantada,[FLAG] Promessa_de_entrega <=5,sac_pos_soma__Avaria em produto entregue,sac_pos_soma__Conferência de Entrega,sac_pos_soma__Entrega errada - inversão,sac_pos_soma__Pedido com alegação de não entregue
0,10,28162012,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,1,0,1,0,0,0,0
1,10,28096157,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,1,0,1,0,0,0,0
2,10,27804324,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,1,0,1,0,0,0,0
3,10,28032244,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,0,0,1,0,1,0,0,0,0
4,10,28214644,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,1,1,1,0,0,0,0


In [40]:
#usar para resetar a base
baseLimpa = base
baseLimpa = baseLimpa.drop(columns=['customer_answer','pv_pai'])
baseLimpa.head()

,Flag Atraso Transporte2_soma,Flag Atraso Fornecedor3_soma,Flag Atraso Coleta4_soma,Flag Atraso Cliente5_soma,[FLAG]Cancelado_soma,[FLAG]Erro prazo transporte validacao_soma,[FLAG]Cross_soma,[FLAG]Atraso_criacao_OC_soma,[FLAG]Reprogramacao_OC_soma,[FLAG]Atraso Fornecedor_soma,...,[FLAG] Private_Label_soma,detrator,neutro,promotor,[FLAG] Entrega_adiantada,[FLAG] Promessa_de_entrega <=5,sac_pos_soma__Avaria em produto entregue,sac_pos_soma__Conferência de Entrega,sac_pos_soma__Entrega errada - inversão,sac_pos_soma__Pedido com alegação de não entregue
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,1,0,1,0,0,0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,1,0,1,0,0,0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,1,0,1,0,0,0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,0,0,1,0,1,0,0,0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,1,1,1,0,0,0,0


In [56]:
col = baseLimpa

colunas = [baseLimpa['Flag Atraso Transporte2_soma'].astype(int),
           baseLimpa['Flag Atraso Fornecedor3_soma'].astype(int)]
       #     ['Flag Atraso Coleta4_soma'],
       #     'Flag Atraso Cliente5_soma',
       #     '[FLAG]Cancelado_soma',
       #     '[FLAG]Erro prazo transporte validacao_soma',
       #     '[FLAG]Cross_soma',
       #     '[FLAG]Atraso_criacao_OC_soma',
       #     '[FLAG]Reprogramacao_OC_soma',
       #     '[FLAG]Atraso Fornecedor_soma']



list = []
for coluna in colunas:
       regressao = sm.OLS(baseLimpa['Flag Atraso Transporte2_soma'].astype(int),coluna).fit()
       r2_transporte2 = regressao.rsquared
       r2_transporte2 = r2_transporte2.astype(str)
       r2_transporte2.append(list)
       
       
       
print(list)
    

AttributeError: 'numpy.str_' object has no attribute 'append'

In [ ]:
dtype

In [12]:
regressao = sm.OLS(Y,X).fit()
r2_transportexfornecedor = regressao.rsquared
r2_transportexfornecedor

0.016646333530718205

In [ ]:
Y = baseLimpa['Flag Atraso Fornecedor3_soma']